# Empire PowerShell UAC Bypass
Detects some Empire PowerShell UAC bypass methods

## Rule Content
```
- title: Empire PowerShell UAC Bypass
  id: 3268b746-88d8-4cd3-bffc-30077d02c787
  status: experimental
  description: Detects some Empire PowerShell UAC bypass methods
  references:
  - https://github.com/EmpireProject/Empire/blob/e37fb2eef8ff8f5a0a689f1589f424906fe13055/data/module_source/privesc/Invoke-EventVwrBypass.ps1#L64
  - https://github.com/EmpireProject/Empire/blob/e37fb2eef8ff8f5a0a689f1589f424906fe13055/data/module_source/privesc/Invoke-FodHelperBypass.ps1#L64
  author: Ecco
  date: 2019/08/30
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - '* -NoP -NonI -w Hidden -c $x=$((gp HKCU:Software\\Microsoft\\Windows Update).Update)*'
      - '* -NoP -NonI -c $x=$((gp HKCU:Software\\Microsoft\\Windows Update).Update);*'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.defense_evasion
  - attack.privilege_escalation
  - attack.t1088
  - car.2019-04-001
  falsepositives:
  - unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*\ \-NoP\ \-NonI\ \-w\ Hidden\ \-c\ $x\=$\(\(gp\ HKCU\:Software\\Microsoft\\Windows\ Update\).Update\)* OR *\ \-NoP\ \-NonI\ \-c\ $x\=$\(\(gp\ HKCU\:Software\\Microsoft\\Windows\ Update\).Update\);*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()